In [48]:
import os 
import glob 
import shutil
import cv2
import sympy
import numpy as np
from PIL import Image
from sympy import symbols
from sympy.matrices import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [49]:
os.chdir('c:\\Users\\QuySech\\OneDrive\\Project\\Singular-Value-Decomposition')
os.getcwd()

'c:\\Users\\QuySech\\OneDrive\\Project\\Singular-Value-Decomposition'

In [50]:
class Directories:
    def __init__(self):
        Folder_Name = ["Images_Folder","Result_Folder"]
        for i in range(len(Folder_Name)):
            if os.path.exists(Folder_Name[i]):
                pass
            else:
                os.mkdir(Folder_Name[i])
        self.Store_Path(os.getcwd(), os.path.join(os.getcwd(),"Images_Folder"), os.path.join(os.getcwd(),"Result_Folder"))                
        self.Convert_to_PNG()
        
    def Store_Path(self, Main_Folder, Path_Images_Folder, Path_Result_Folder):
        self.Main_Folder = Main_Folder
        self.Images_Folder = Path_Images_Folder
        self.Result_Folder = Path_Result_Folder
   
    def Convert_to_PNG(self):
        Image_list = ["*.png", "*.jpeg","*.jpg"]        
        for i in range(len(Image_list)):
            Image_File_Raster = glob.glob(Image_list[i])            
            for n in range(len(Image_File_Raster)):
                if ".png" in Image_File_Raster[n]:
                    shutil.move(Image_File_Raster[n], "Images_Folder")
                else:
                    Image_name = Image_File_Raster[n][::-1].split(".",1)
                    Renamed = Image_name[1][::-1]+".png"
                    os.rename(Image_File_Raster[n],Image_name[1][::-1]+".png")
                    shutil.move(Renamed, "Images_Folder")                      
Path = Directories()
os.chdir(Path.Main_Folder)

In [52]:
def Convert_Image_To_Matrix():
  os.chdir(Path.Images_Folder)
  picture_name = "sample.png"
  if picture_name in os.listdir():
    print("Found the picture:", picture_name)
    img = cv2.imread(picture_name)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    data = np.array(gray_img)/255
    return data
  else:
    print("Picture not found.")     
Convert_Image_To_Matrix()

Found the picture: sample.png


array([[0.10980392, 0.10588235, 0.1372549 , ..., 0.1372549 , 0.18431373,
        0.18431373],
       [0.        , 0.08235294, 0.15294118, ..., 0.12941176, 0.21960784,
        0.21960784],
       [0.05098039, 0.2       , 0.25098039, ..., 0.22352941, 0.1254902 ,
        0.1254902 ],
       ...,
       [0.36862745, 0.36862745, 0.36862745, ..., 0.36862745, 0.36862745,
        0.36862745],
       [0.36862745, 0.36862745, 0.36862745, ..., 0.36862745, 0.36862745,
        0.36862745],
       [0.36862745, 0.36862745, 0.36862745, ..., 0.36862745, 0.36862745,
        0.36862745]])

In [53]:
def sort(eigvaule_vector):
    dict = {}
    for x in range(len(eigvaule_vector[0])):
        dict[eigvaule_vector[0][x]] = eigvaule_vector[1][:, x]
    key = list(dict.keys())
    key = sorted(key, key=None, reverse=True)
    return {i: dict[i] for i in key}

In [54]:
def find_eigenvalue(matrix):
    x = symbols('x')
    fun =matrix-x*sympy.eye(len(matrix[0]))
    return sympy.solve(sympy.det(fun))

def find_eigenvector(matrix):
  arr = []
  w,v = np.linalg.eigh(matrix)
  # print(w,v)
  return w,v


In [55]:
def show_image(result,k):
  os.chdir(Path.Result_Folder)
  picture_name = "sample.png"
  img_array = np.array(result, dtype=np.uint8)
  img = Image.fromarray(img_array)
  print(img_array.shape)
  if picture_name in os.listdir():
    img = cv2.imread(picture_name)

  new_location = Path.Result_Folder
  new_filename = f"aFinal_{k}.jpg"

  # Create the new folder if it doesn't exist
  if not os.path.exists(new_location):
    os.makedirs(new_location)

  # Save the image to the new location
  cv2.imwrite(os.path.join(new_location, new_filename), img_array)
  

In [56]:
os.chdir(Path.Main_Folder)
os.getcwd()

'c:\\Users\\QuySech\\OneDrive\\Project\\Singular-Value-Decomposition'

In [57]:
class math:
    def __init__(self):
        self.matrix = Convert_Image_To_Matrix()
        # self.matrix = np.array([[1,0,1],[0,1,1]])
        self.shape = np.shape(self.matrix)
        self.k = 5
        result = self.SVD()
        show_image(result*255,self.k)


    def SVD(self):
      c,r = self.shape
      A = self.matrix
      V = []
      U = []
      print(np.shape(self.matrix))
      AtA = np.matmul(np.transpose(self.matrix),self.matrix)

      b = find_eigenvector(np.array(AtA,dtype= float))

      """turn into dict then soft in descending"""
      Newdict = sort(b)

      """now find UDV.T"""
      D = np.zeros(self.shape,dtype = 'complex_')
      r,c = self.shape
      for x in range(min(r,c)):
          D[x][x] = np.sqrt(np.abs(list(Newdict.keys())[x]))
      d = D[0:self.k,0:self.k]


      for x in Newdict.values():
          V.append(list(x))
      V = Matrix(V).T
      v = V[:,0:self.k]

      for i in range(min(self.k,r)):
        U.append((np.matmul(A,V[:,i])/D[i][i]).T[0])
      U = Matrix(U).T
      u = U[:, 0:self.k]

      # for full information preserve, find A instead
      # A = U*D*(V.T)

      # print('D = ',Matrix(d))
      # print('V =',V)
      # print('U = ',U)
      # # print('A =', A)
      # return A

      a = u * d * (v.T)

      print('D = ',Matrix(d))
      print('V =',v)
      print('U = ',u)
      print('A =', a) #overflow google presentaion anyway

      return a


if __name__ == '__main__':
    math()

Found the picture: sample.png
(283, 178)
D =  Matrix([[70.3200940101193, 0, 0, 0, 0], [0, 13.4393890739300, 0, 0, 0], [0, 0, 9.12330035021496, 0, 0], [0, 0, 0, 8.29922583246754, 0], [0, 0, 0, 0, 7.42080083979529]])
V = Matrix([[0.0853532842455621, 0.0129912438241001, 0.0429761357436136, -0.00709008162282106, 0.131581889263681], [0.0871692139802195, 0.0334764992120649, 0.0324563345007045, 0.0112503501338162, 0.106850213309603], [0.0887954480546420, 0.0321241917975350, 0.0356209528343140, 0.00189309313660344, 0.124227482672483], [0.0901030589330732, -0.00214529561754662, 0.0473842680886327, 0.00543915693731560, 0.111188457936941], [0.0881762859617011, 0.0121643753515046, 0.0582677575697557, 0.0210310082873176, 0.0982842070155989], [0.0931239485526729, 0.0143599834705020, 0.0525592022183954, 0.0216672831089704, 0.104294609157849], [0.0868559063535466, 0.00917767247232268, 0.0213631932632923, 0.00258836224204168, 0.0935849034855688], [0.0938691704519082, 0.0183794851901730, 0.0415555546414